```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Basic Particle Filter

In this notebook we will make test implementations of basic particle filters.

The aim is to find a decent implementation of the particles, which can be used by both simulators and particle filter.

All post-processing of particles will be done on the CPU in this first iteration.

## Overview of basic resampling strategies

The following four resampling algorithms are described in Section 3a) in van Leeuven's 2009 review paper.

The starting point is a prior distribution of model states with pdf $p(\psi^0)$, from which $N$ model state samples (particles) $\psi_i^0$, $i = 1,...,N$ are drawn.
Run the simulation model on all particles $\psi_i^n = f(\psi_i^{n-1})$. This is the same as sampling from the $p(\psi^n | \psi_i^{n-1})$.
At this point we see an observation $d$.

Now, define a posterior distribution $p(\psi^n | d)$. **Think a bit here before writing more** - from this we obtain weights $w_i$.


The most basic resampling strategies are as follows:
### Probabilistic resampling
Use the weights as a discrete distribution and sample directly from this.

### Residual sampling
Here, we first resample particles deterministic based on their weights. Resample particle $i$ `np.floor`$(Nw_i)$ times. Define the left-over weights as $w^*_i = Nw_i \% 1$, and use $w^*_i$ as a discrete distribution, from which the reminder resampled particles are drawn from, until we have $N$ particles again.

**Note:** It is here recommended to use Cauchy distribution as the observation pdf.

### Stochastic Universal sampling
Put all weights as buckets on the line $[0, 1]$, and draw a random number $u \sim U[0, 1/N]$.
Put $N$ line pieces starting from $u$ with length $1/N$ are laid on the line $[0,1]$. 
The bucket in which each line piece ends (not started???) defines which particle is chosen for each of the $N$ line pieces.

### Monte Carlo Metropolis-Hastings sampling
This sampling scheme is described very algorithmic in the paper already, so take a look at 3a4).


#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec


import os
import pyopencl
import datetime
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import CTCS, CDKLM16, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *


In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

## Thoughts on code structure

The observation will in these initial cases be a chosen model realization. When initializing a data assimilation with N particles, N+1 particles should be created and distributed on simulators.

One hypothesis for our ocean simulator is that integrating 100 particles within the same simulation is equally expensive as integrating 1 particle. Each particle integration should be done with a single thread on the GPU, so all 100 particles will can be processed in parallel.

If this assumption is true, it is best to have all particle positions continuous in memory. Hence, it will be implemented as an struct of array.

## Create random particles, and create random observation

Particles are created in a GlobalParticle class, which holds the positions of all ensemble member particles, and one additional particle which serves as our observation.
This class should have all computational functionality that relies on the relationships between particles and the observation, and the ensemble itself.

Filtering and resampling of particles does not belong in this class.

List of functions:
- [x] initialize uniform on unit square
- [ ] initialize gaussian 
- [ ] Calculate distances from observation
- [x] get weights from Gaussian distribution
- [x] get weights from Cauchy distribution
- [ ] find ensemble mean position
- [ ] find ensemble variance
- [ ] set observation to a given coordinate

**About distances**: In order to calculate distances in a unified way, information about boundary conditions needs to be known by the class. E.g., the distance between a particle at $(0.99, 0.99)$ from an observation at $(0.01, 0.02)$ on a unit square domain, is about $\sqrt{2}$. However, with periodic boundary conditions, their distance is only $0.05$.

Let the domain size be $(L_x, L_y)$, and define the particle and observation positions as $(x_p, y_p)$ and $(x_o, y_o)$, respectively.
The minimal distance with a periodic boundary can then be found by
$$ d_{x,min} = \min \left\{ |x_p - x_o|, |(x_p - L_x) - x_o|, |(x_p + L_x) - x_o| \right\}$$
$$ d_{y,min} = \min \left\{ |y_p - y_o|, |(y_p - L_y) - y_o|, |(y_p + L_y) - y_o| \right\}$$
and 
$$ d_{min} = \sqrt{ d_{x,min}^2 + d_{y,min}^2}$$

** About ensemble mean position**: When finding the mean position of particles, the above considerations needs to be taken as well. The ensemble mean position should be found by the coordinate position which results in the minimal distance given above.
In other words, the position we should consider for the mean is
$$ x^*_p = {\arg\min}_{x \in \{x_p, x_p \pm L_x \}} |x - x_o|, $$
$$ y^*_p = {\arg\min}_{y \in \{y_p, y_p \pm L_y \}} |y - y_o|. $$

In [ ]:
# Seed so that all simulation runs are equal.
np.random.seed(10)

print np.zeros((2,3))
a = np.random.rand(3,2)
print a, a.shape

In [ ]:
### GlobalParticlesClass

class GlobalParticles:
    def __init__(self, numParticles, observation_variance=0.1, boundaryConditions=Common.BoundaryConditions()):
        
        self.numParticles = numParticles
        
        # Observation index is the last particle
        self.obs_index = self.numParticles 
        self.observation_variance = observation_variance
        
        # One position for every particle plus observation
        self.positions = np.zeros((self.numParticles + 1, 2))
        
        self.domain_size_x = 1.0
        self.domain_size_y = 1.0
        
        # Boundary conditions are read from a BoundaryConditions object
        self.boundaryConditions = boundaryConditions
        
    def initializeInSquare(self, domain_size_x=1.0, domain_size_y=1.0):
        
        # Initialize in unit square
        self.positions = np.random.rand(self.numParticles + 1, 2)
        # Ensure that the observation is in the middle 0.5x0.5 square:
        self.positions[self.obs_index, :] = self.positions[self.obs_index]*0.5 + 0.25
        
        # Map to given square
        self.positions[:,0] = self.positions[:,0]*domain_size_x
        self.positions[:,1] = self.positions[:,1]*domain_size_y
        
        self.domain_size_x = domain_size_x
        self.domain_size_y = domain_size_y
        
    """
    Returns a set of coordinates corresponding to each particles closest position to the observation,
    considering possible periodic boundary conditions
    """
    def _getClosestPositions(self):
        if not (self.boundaryConditions.isPeriodicNorthSouth() or self.boundaryConditions.isPeriodicEastWest()):
            return self.positions
        else:
            periodicPositions = self.positions.copy()
            obs_x, obs_y = periodicPositions[self.obs_index, :]
            if self.boundaryConditions.isPeriodicEastWest():
                for i in range(self.numParticles):
                    x = periodicPositions[i,0]
                    
                    pos_x = np.array([x - self.domain_size_x, x, x + self.domain_size_x])
                    dist_x = np.abs(pos_x - obs_x)
                    periodicPositions[i,0] = pos_x[np.argmin(dist_x)]

            if self.boundaryConditions.isPeriodicNorthSouth():
                for i in range(self.numParticles):
                    y = periodicPositions[i,1]
                    
                    pos_y = np.array([y - self.domain_size_y, y, y + self.domain_size_y])
                    dist_y = np.abs(pos_y - obs_y)
                    periodicPositions[i,1] = pos_y[np.argmin(dist_y)]
        return periodicPositions
        
        
    def getDistances(self):
        distances = np.zeros(self.numParticles)
        closestPositions = self._getClosestPositions()
        obs_x, obs_y = self.positions[self.obs_index, :]
        for i in range(self.numParticles):
            distances[i] = np.sqrt( (closestPositions[i,0]-obs_x)**2 +
                                    (closestPositions[i,1]-obs_y)**2)
        return distances
        
    def getParticlePositions(self):
        return self.positions[:-1,:]
    
    def getObservationPosition(self):
        return self.positions[-1, :]
    
    def getGaussianWeight(self, distance=None, normalize=True):
        if distance is None:
            distance = self.getDistances()
        weights = (1.0/np.sqrt(2*np.pi*self.observation_variance**2))* \
            np.exp(- (distance**2/(2*self.observation_variance**2)))
        if normalize:
            return weights/np.sum(weights)
        return weights
    
    """
    Weights are calculated using a Cauchy Distribution.
    It is chosen over a Gauss distribution in order to obtain wider tails.
    """
    def getCauchyWeight(self, distance=None, normalize=True):
        if distance is None:
            distance = self.getDistances()
        weights = 1.0/(np.pi*self.observation_variance*(1 + (distance/self.observation_variance)**2))
        if normalize:
            return weights/np.sum(weights)
        return weights    
    
    def plotDistanceInfo(self, title=None):
        fig = plt.figure(figsize=(10,6))
        gridspec.GridSpec(2, 3)
        
        # PLOT POSITIONS OF PARTICLES AND OBSERVATIONS
        ax0 = plt.subplot2grid((2,3), (0,0))
        plt.plot(self.getParticlePositions()[:,0], \
                 self.getParticlePositions()[:,1], 'b.')
        plt.plot(self.getObservationPosition()[0], \
                 self.getObservationPosition()[1], 'r.')
        plt.xlim(0, self.domain_size_x)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.ylim(0, self.domain_size_y)
        plt.title("Particle positions")
        
        # PLOT DISCTRIBUTION OF PARTICLE DISTANCES AND THEORETIC OBSERVATION PDF
        ax0 = plt.subplot2grid((2,3), (0,1), colspan=2)
        distances = self.getDistances()
        plt.hist(distances, bins=30, range=(0, max(min(self.domain_size_x, self.domain_size_y), np.max(distances))),\
                 normed=True, label="particle distances")
        
        # With observation 
        x = np.linspace(0, max(self.domain_size_x, self.domain_size_y), num=100)
        cauchy_pdf = self.getCauchyWeight(x, normalize=False)
        gauss_pdf = self.getGaussianWeight(x, normalize=False)
        plt.plot(x, cauchy_pdf, 'r', label="obs Cauchy pdf")
        plt.plot(x, gauss_pdf, 'g', label="obs Gauss pdf")
        plt.legend()
        plt.title("Distribution of particle distances from observation")
        
        # PLOT SORTED DISTANCES FROM OBSERVATION
        ax0 = plt.subplot2grid((2,3), (1,0), colspan=3)
        cauchyWeights = self.getCauchyWeight(distances)
        gaussWeights = self.getGaussianWeight(distances)
        indices_sorted_by_observation = distances.argsort()
        plt.plot(distances[indices_sorted_by_observation], label="distance")
        plt.plot(cauchyWeights[indices_sorted_by_observation]/np.max(cauchyWeights), 'r', label="Cauchy weight")
        plt.plot(gaussWeights[indices_sorted_by_observation]/np.max(gaussWeights), 'g', label="Gauss weight")
        plt.title("Sorted distances from observation")
        plt.grid()
        plt.ylim(0,1.4)
        plt.legend()
        if title is not None:
            plt.suptitle(title, fontsize=16)

# Initialize an ensemble of particles:
N = 100
observation_variance = 0.08
resample_variance = 0.005
bc = 2
boundaryConditions = Common.BoundaryConditions(bc,bc,bc,bc)
globalParticles = GlobalParticles(N, observation_variance, boundaryConditions=boundaryConditions)
globalParticles.initializeInSquare()

# The particles are by now drawn from the prior distribution

# Here, the simulation/time integration should take place

# Inspect initial ensemble
globalParticles.plotDistanceInfo(title="Initial particles")
print "Observation: ", globalParticles.getObservationPosition()


In [ ]:
## Testing boundary conditions
test_N = 3
far   = "far      "
close = "close    "
test_solutions = np.array([[[far, far, far], [far, close, far]], [[far, far, close], [close, close, close]]])
for test_bc_ns in [1, 2]:
    for test_bc_ew in [1, 2]:
        test_boundaryConditions = Common.BoundaryConditions(test_bc_ns, test_bc_ew, test_bc_ns, test_bc_ew)
        test_particles = GlobalParticles(test_N, boundaryConditions=test_boundaryConditions)
        test_particles.positions[0,:] = [0.9, 0.9]
        test_particles.positions[1,:] = [0.9, 0.1]
        test_particles.positions[2,:] = [0.1, 0.9]
        test_particles.positions[3,:] = [0.1, 0.1]
        print "(test_bc_ns, test_bc_ew)", (test_bc_ns, test_bc_ew)
        print test_particles.getDistances()
        print test_solutions[test_bc_ns-1, test_bc_ew-1, :]
        print "--------------------"
print test_particles.positions
#test_particles.plotDistanceInfo()

## Probabilistic Resampling

In [ ]:
globalParticles.plotDistanceInfo(title="Initial particles")

# Create a new GlobalParticles instance based on newSampleIndices
def resampleParticles(particles, newSampleIndices, reinitialization_variance):
    newParticles = GlobalParticles(len(newSampleIndices))
    if particles.numParticles != newParticles.numParticles:
        print "WARNING: The size of the new ensemble differs from the old size!"
        print "(old size, new size): ", (particles.numParticles, newParticles.numParticles)
    
    # We really do not the if. The random number with zero variance returns exactly the mean
    if reinitialization_variance == 0:
        # Simply copy the given positions
        newParticles.positions[:-1,:] = particles.positions[newSampleIndices,:].copy()
    else:
        # Make sure to make a clean copy of first resampled particle, and add a disturbance of the next ones.
        resampledOnce = np.full(particles.numParticles, False, dtype=bool)
        var = np.eye(2)*reinitialization_variance
        for i in range(len(newSampleIndices)):
            index = newSampleIndices[i]
            if resampledOnce[index]:
                newParticles.positions[i,:] = np.random.multivariate_normal(particles.positions[index,:], var)
            else:
                newParticles.positions[i,:] = particles.positions[index,:]
                resampledOnce[index] = True
                                                                            
        
        
    # Copy the observation:
    newParticles.positions[-1,:] = particles.positions[-1,:]
    
    return newParticles
    


def probabilisticResampling(particles, reinitialization_variance=0):
    # Obtain weights:
    weights = particles.getGaussianWeight()
    #weights = particles.getCauchyWeight()
    
    # Create array of possible indices to resample:
    allIndices = range(particles.numParticles)
    
    # Draw new indices based from discrete distribution based on weights
    newSampleIndices = np.random.choice(allIndices, particles.numParticles, p=weights)
        
    # Return a new set of particles
    return resampleParticles(particles, newSampleIndices, reinitialization_variance)

probabilisticResampledParticles = probabilisticResampling(globalParticles, reinitialization_variance=0.0)
probabilisticResampledParticles.plotDistanceInfo(title="From probabilistic resampling (identical resampling)")

probabilisticResampledParticles = probabilisticResampling(globalParticles, reinitialization_variance=resample_variance)
probabilisticResampledParticles.plotDistanceInfo(title="From probabilistic resampling (gaussian resampling)")

## Residual Sampling

In [ ]:
 
def residualSampling(particles, reinitialization_variance=0, onlyDeterministic=False, onlyStochastic=False):
    # Obtain weights:
    #weights = particles.getCauchyWeight()
    weights = particles.getGaussianWeight()

    # Create array of possible indices to resample:
    allIndices = range(particles.numParticles)

    # Deterministic resampling based on the integer part of N*weights:
    weightsTimesN = weights*particles.numParticles
    weightsTimesNInteger = np.int64(np.floor(weightsTimesN))
    deterministicResampleIndices = np.repeat(allIndices, weightsTimesNInteger)
    
    # Stochastic resampling based on the decimal parts of N*weights:
    decimalWeights = np.mod(weightsTimesN, 1)
    decimalWeights = decimalWeights/np.sum(decimalWeights)
    stochasticResampleIndices = np.random.choice(allIndices, 
                                                 particles.numParticles - len(deterministicResampleIndices), 
                                                 p=decimalWeights)
    ### NOTE!
    # In numpy v >= 1.13, np.divmod can be used to get weightsTimesNInteger and decimalWeights from one function call.
    
    if onlyDeterministic:
        return resampleParticles(particles, deterministicResampleIndices, reinitialization_variance)
    if onlyStochastic:
        return resampleParticles(particles, stochasticResampleIndices, reinitialization_variance)
    
    return resampleParticles(particles, np.concatenate((deterministicResampleIndices, stochasticResampleIndices)), \
                             reinitialization_variance)
    
globalParticles.plotDistanceInfo(title="Initial particles")



residualSamplingParticles = residualSampling(globalParticles, reinitialization_variance=resample_variance)
residualSamplingParticles.plotDistanceInfo(title="From residual sampling")

residualSamplingParticlesDet = residualSampling(globalParticles, reinitialization_variance=resample_variance, \
                                                onlyDeterministic=True)
residualSamplingParticlesDet.plotDistanceInfo(title="From residual sampling - Deterministic part only")

residualSamplingParticlesStoc = residualSampling(globalParticles, reinitialization_variance=resample_variance,\
                                                 onlyStochastic=True)
residualSamplingParticlesStoc.plotDistanceInfo(title="From residual sampling - Stochastic part only")


## Stochastic Universal Sampling

In [ ]:
def stochasticUniversalSampling(particles, reinitialization_variance=0):
    # Obtain weights:
    #weights = particles.getCauchyWeight()
    weights = particles.getGaussianWeight()

    # Create array of possible indices to resample:
    allIndices = np.array(range(particles.numParticles))
    
    # Create histogram buckets based on the cumulative weights
    cumulativeWeights = np.concatenate(([0.0], np.cumsum(weights)))
    
    # Find first starting position:
    startPos = np.random.rand()/particles.numParticles
    lengths = 1.0/particles.numParticles
    #print startPos, lengths
    selectionValues = allIndices*lengths + startPos
    
    # Create a histogram of selectionValues within the cumulativeWeights buckets
    bucketValues, buckets = np.histogram(selectionValues, bins=cumulativeWeights)
    
    #newSampleIndices has now the number of times each index should be resampled
    # We need to go from [0, 0, 1, 4, 0] to [2,3,3,3,3]
    newSampleIndices = np.repeat(allIndices, bucketValues)
    
    # Return a new set of particles
    return resampleParticles(particles, newSampleIndices, reinitialization_variance)

    
globalParticles.plotDistanceInfo(title="Initial particles")

stochasticUniversalParticles = stochasticUniversalSampling(globalParticles, reinitialization_variance=resample_variance)
stochasticUniversalParticles.plotDistanceInfo(title="From Stochastic universal sampling")


## Monte Carlo Metropolis-Hasting

In [ ]:
def mcMetropolisHastingSampling(particles,  reinitialization_variance=0):
    # Obtain weights:
    #weights = particles.getCauchyWeight()
    weights = particles.getGaussianWeight()
    
    # Create buffer for indices which should be in the new ensemble:
    newSampleIndices = np.zeros_like(weights, dtype=int)
    
    # The first member is automatically a member of the new ensemble
    newSampleIndices[0] = 0
    
    # Iterate through all weights, and apply the Metropolis-Hasting algorithm
    for i in range(1, particles.numParticles):
        # Draw random number U[0,1]
        p = np.random.rand()
        if p < weights[i]/weights[newSampleIndices[i-1]]:
            newSampleIndices[i] = i
        else:
            newSampleIndices[i] = newSampleIndices[i-1]
    
    # Return a new set of particles
    return resampleParticles(particles, newSampleIndices, reinitialization_variance)

    
globalParticles.plotDistanceInfo(title="Initial particles")

metropolisHastingParticles = mcMetropolisHastingSampling(globalParticles, reinitialization_variance=resample_variance)
metropolisHastingParticles.plotDistanceInfo(title="From Monte Carlo Metropolis-Hasting")

    

# Naïve drift trajectories in the SWE simulators

Here, we will make a naive implementation of particles drifting within our simplified ocean models.
For simplicity, a non-staggered implementation is chosen, as it makes it easier to evaluate the velocity field.


In [ ]:
# DEFINE PARAMETERS

#Coriolis well balanced reconstruction scheme
nx = 50
ny = 50

dx = 4.0
dy = 4.0

dt = 0.1
g = 9.81

f = 0.5
r = 0.0

waterHeight = 10

# WIND
wind = Common.WindStressParams(type=99)

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

# Define which cell index which has lower left corner as position (0,0)
x_zero_ref = 2
y_zero_ref = 2

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')*waterHeight

# Add disturbance:
addBump(eta0, nx, ny, dx, dy, 0.3, 0.5, 0.05, validDomain)
addBump(eta0, nx, ny, dx, dy, 0.7, 0.2, 0.10, validDomain)
addBump(eta0, nx, ny, dx, dy, 0.1, 0.8, 0.03, validDomain)
eta0 = eta0*0.3

#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
#x_coords = np.subtract(x_coords, x_center)
#y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))



In [ ]:
## Define a bunch of particles to be released within the given domain
numParticles = 50
observation_variance = 5*dx

constOceanParticles = GlobalParticles(numParticles, observation_variance)
constOceanParticles.initializeInSquare(dx*nx, dy*ny)

constOceanParticles.plotDistanceInfo(title="Initial particles")

In [ ]:
def applyPeriodicBoundaryConditionsToParticles(positions, nx, ny, dx, dy):
    # Loop over particles
    for i in range(positions.shape[0]):
        x, y = positions[i,0], positions[i,1]
        
        # Check what we assume is periodic boundary conditions
        if x < 0:
            x = dx*nx + x
        if y < 0:
            y = dy*ny + y
        if x > dx*nx:
            x = dx*nx - x
        if y > dy*ny:
            y = dy*ny - y
        
        positions[i,0] = x
        positions[i,1] = y

def particleDrifter(positions, eta, hu, hv, H0, dt, nx, ny, dx, dy, x_zero_ref, y_zero_ref, \
                    sensitivity=1, doPrint=False):
    # Change positions by reference
    
    numParticles = positions.shape[0]
    # Loop over particles
    for i in range(numParticles):
        if doPrint: print "---------- Particle " + str(i) + " ---------------"
        x0, y0 = positions[i,0], positions[i,1]
        if doPrint: print "(x0, y0): ", (x0,y0)
        
        # First, find which cell each particle is in
        
        # In x-direction:
        cell_id_x = int(np.ceil(x0/dx) + x_zero_ref)
        cell_id_y = int(np.ceil(y0/dy) + y_zero_ref)
        
        if doPrint: print "cell values in x-direction: ", ((cell_id_x-2-0.5)*dx, (cell_id_x-2+0.5)*dx)
        if doPrint: print "cell values in y-direction: ", ((cell_id_y-2-0.5)*dy, (cell_id_y-2+0.5)*dy)
        
        h = waterHeight + eta[cell_id_y, cell_id_x]
        u = hu[cell_id_y, cell_id_x]
        v = hv[cell_id_y, cell_id_x]
        
        if doPrint: print "Velocity: ", (u, v)
        
        x1 = sensitivity*u*dt + x0
        y1 = sensitivity*v*dt + y0
        if doPrint: print "(x1, y1): ", (positions[i,0], positions[i,1])
        
        positions[i,0] = x1
        positions[i,1] = y1
        
    
    # Check what we assume is periodic boundary conditions    
    applyPeriodicBoundaryConditionsToParticles(positions, nx, ny, dx, dy)
        
        
#eta1, hu1, hv1 = sim.download()
#particleDrifter(oceanParticles.positions, eta1, hu1, hv1, waterHeight, \
#                dt, nx, ny, dx, dy, x_zero_ref, y_zero_ref, \
#                doPrint=True)

In [ ]:

oceanParticles = GlobalParticles(numParticles)
oceanParticles.positions = constOceanParticles.positions.copy()

#Clean up old simulator if any:
if 'sim' in globals():
    sim.cleanUp()
    
#Initialize simulator
reload(CDKLM16)
reload(PlotHelper)
sim = CDKLM16.CDKLM16(cl_ctx, eta0, u0, v0, Hi, \
                nx, ny, dx, dy, dt, g, f, r, \
                boundary_conditions=boundaryConditions)

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

plotter.showParticles(oceanParticles.getParticlePositions(),
                      oceanParticles.getObservationPosition())

T = 200
sensitivity = 20
loopsPerFrame = 10
def animate(i):
    if (i>0):
        for j in range(loopsPerFrame):
            t = sim.step(dt)
            eta1, hu1, hv1 = sim.download()

            particleDrifter(oceanParticles.positions, eta1, hu1, hv1, waterHeight, \
                            dt, nx, ny, dx, dy, x_zero_ref, y_zero_ref, sensitivity=sensitivity)

    else:
        t = 0.0

    eta1, hu1, hv1 = sim.download()
    plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 hu1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 hv1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    
    plotter.showParticles(oceanParticles.getParticlePositions(),
                          oceanParticles.getObservationPosition())
    
    fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax eta: " + str(np.max(eta1)) + \
        "\tMax hu: " + str(np.max(hu1)) + \
        "\tMax hv: " + str(np.max(hv1))
        print "\t\tObservation pos: ", oceanParticles.getObservationPosition()
                     
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
print constOceanParticles.getObservationPosition()
print oceanParticles.getObservationPosition()


In [ ]:
fig = plt.figure(figsize=(4,4))
posis = np.array([[0.1, 0.2, 0.3, 0.4],[0.2, 0.4, 0.6, 0.8]])
print posis
scat = plt.scatter(x=posis[0,:], y=posis[1,:])
plt.xlim(0,1)
plt.ylim(0,1)
plt.grid()
posis[0,:] = posis[0,:]*1.8
print posis
newData = np.hstack((xpath*2, ypath))
scat.set_offsets(posis.T)


In [ ]:
#### Sorting example from https://stackoverflow.com/a/21077060

people = np.array(['Jim', 'Pam', 'Micheal', 'Dwight'])
ages = np.array([27, 25, 4, 9])
sorted_indices = ages.argsort()
print people[sorted_indices]